## API - Carregamento, limpeza, transformação e Insere db pentalupop
##### coding: utf-8
##### @Tiago de Camargo::
### github - https://github.com/ticamargo
### linkedin - https://www.linkedin.com/in/tiagodecamargo/

In [ ]:
## IMPORTACAO DE PACOTES
import xml.etree.ElementTree as et
from unicodedata import normalize
from datetime import timedelta, date
import pandas as pd
import cx_Oracle
import requests
import warnings

In [ ]:
## MOSTRAR COLUNAS DO DATAFRAME! FACILITA A VISUALIZACAO
pd.set_option('display.max_columns', 80)
warnings.simplefilter('ignore')

In [ ]:
## IMPORTAR DATA DO SISTEMA 
hoje = date.today()
print('Data de Hoje: '+str(hoje))

intervalo_1d = timedelta(1)
ontem = str(hoje - intervalo_1d)
print('Data de Ontem: '+str(ontem))

intervalo_10d = timedelta(10)
dt_10d = hoje - intervalo_10d
print('Data de 10 dias: '+str(dt_10d))

tempdias_10 = str(dt_10d)
print(tempdias_10)
dias_10 = tempdias_10.replace('-', '')
print(dias_10)

tempontem_10 = str(ontem)
print(tempontem_10)
ontem_10 = tempontem_10.replace('-', '')
print(ontem_10)

In [ ]:
## ALGUMAS VARIAVEIS
url = "URL DA API"
lista_cnpj, listacnpj_temp, dados_cnpj, df_cnpj, ncols_cnpj  = [],[],[],[],[]
filtro_tag = "D"

In [ ]:
dias_10 = '20190927'
print(dias_10)
ontem_10 = '20190927'
print(ontem_10)

##### INICIO - CONSUMO DAS LOJAS QUE SERÃO CONSULTADAS

In [ ]:
## CONSULTANDO API DE GRUPOLOJAS - LISTA TODAS AS INFORMAÇÕES DE LOJAS
payload_lojas = "<?xml version='1.0' encoding='ISO-8859-1'?> \
<LinxMicrovix> \
<Authentication user='USER' password='PASS'/> \
<ResponseFormat>xml</ResponseFormat> \
<Command> \
<Name>LinxGrupoLojas</Name> \
<Parameters> \
<Parameter id='chave'>TOKEN</Parameter> \
<Parameter id='grupo'>REDE DE LOJAS</Parameter> \
</Parameters> \
</Command> \
</LinxMicrovix>"
response = requests.request("POST", url, data=payload_lojas)
root_cnpj = et.fromstring(response.content)

In [ ]:
## EXTRAIR NUMERO DE COLUNAS E NOME DAS COLUNAS DA API LINXGRUPOLOJAS
for child in root_cnpj.iter('*'):
    #print(child.tag, child.text)
    if child.tag == 'C':
        ncols_cnpj = len(child)
        #print(len(child))
        #print(ncols_cnpj)

In [ ]:
## TRANSFORMAR EM OBJETO ELEMENT TREE
for child in root_cnpj.iter(filtro_tag):
    listacnpj_temp.append(child.text)

In [ ]:
df_cnpj = listacnpj_temp[0:ncols_cnpj]
#print(df_cnpj)

In [ ]:
## QUEBRAR EM SUBLISTAS COM TAMANHO DAS COLUNAS N
for i in range(ncols_cnpj, len(listacnpj_temp), ncols_cnpj):
    dados_cnpj.append(listacnpj_temp[i: i + ncols_cnpj])

In [ ]:
#dados_cnpj[8:10]

In [ ]:
## CRIAR DATAFRAME PARA ORGANIZAR E LIMPAR OS DADOS
dataframe_cnpj = pd.DataFrame(dados_cnpj, columns=df_cnpj)

In [ ]:
#dataframe_cnpj.head(10)

In [ ]:
## ELIMINAR CNPJS DUPLICADOS VINDOS DA API LINXGRUPOLOJAS
dataframe_cnpj.drop_duplicates(['CNPJ'], keep="last", inplace=True)
dataframe_cnpj = dataframe_cnpj.reset_index(drop=True)

In [ ]:
#dataframe_cnpj.head()

In [ ]:
## CRIAR LISTA COM OS CNPJS
for i in dataframe_cnpj.index:
    if dataframe_cnpj.loc[i, 'CNPJ'] != None:
        lista_cnpj.append(dataframe_cnpj.loc[i,'CNPJ'])

In [ ]:
lista_cnpj[0:5]

In [ ]:
## EXPORTAR ARQUIVO CSV COM OS DADOS CONSUMIDOS DA API
dataframe_cnpj.to_csv('APIGrupoLojas.csv', index=False)

#### FIM - CONSUMO DAS LOJAS QUE SERÃO CONSULTADAS

#### INICIO - CONSUMO DOS CUPONS DE VENDAS DAS LOJAS

In [ ]:
## VARIAVEIS
df_mov = []
lista_mov = []
ncols_mov = []
dados_mov = []

In [ ]:
## ENTRADA MANUAL DAS DATAS PARA TESTE
#ontem = '2019-09-24'
#hoje = '2019-09-24'
print('10 dias anteriores: '+str(dt_10d))
print('Ontem: '+ontem)
print('Hoje: '+str(hoje))

In [ ]:
## CHAMADA PARA CONSUMIR API
for i in range(len(lista_cnpj)):
    cnpj = lista_cnpj[i].rstrip()
    print(str(i+1) + ', '+ str(lista_cnpj[i]) +'')
    payload = "<?xml version='1.0' encoding='ISO-8859-1'?> \
    <LinxMicrovix> \
    <Authentication user='USER' password='PASS'/> \
    <ResponseFormat>xml</ResponseFormat> \
    <Command> <Name>LinxMovimento</Name> \
    <Parameters> \
    <Parameter id='chave'>TOKEN</Parameter> \
    <Parameter id='cnpjEmp'>"+cnpj+"</Parameter> \
    <Parameter id='data_inicial'>"+str(dt_10d)+"</Parameter> \
    <Parameter id='data_fim'>"+str(ontem)+"</Parameter> \
    </Parameters> \
    </Command> \
    </LinxMicrovix>"
    response = requests.request("POST", url, data=payload)
    root = et.fromstring(response.content)
    for child in root.iter(filtro_tag):
        lista_mov.append(child.text)

In [ ]:
## PARA TESTAR!!!
## CHAMADA PARA CONSUMIR API
for i in range(len(lista_cnpj)):
    cnpj = lista_cnpj[i].rstrip()
    print(str(i+1) + ', '+ str(lista_cnpj[i]) +'')
    payload = "<?xml version='1.0' encoding='ISO-8859-1'?> \
    <LinxMicrovix> \
    <Authentication user='USER' password='PASS'/> \
    <ResponseFormat>xml</ResponseFormat> \
    <Command> <Name>LinxMovimento</Name> \
    <Parameters> \
    <Parameter id='chave'>TOKEN</Parameter> \
    <Parameter id='cnpjEmp'>"+cnpj+"</Parameter> \
    <Parameter id='data_inicial'>2019-10-01</Parameter> \
    <Parameter id='data_fim'>2019-10-16</Parameter> \
    </Parameters> \
    </Command> \
    </LinxMicrovix>"
    response = requests.request("POST", url, data=payload)
    root = et.fromstring(response.content)
    for child in root.iter(filtro_tag):
        lista_mov.append(child.text)
## PARA TESTAR!!

In [ ]:
lista_mov[78:156]

In [ ]:
## EXTRAIR NUMERO DE COLUNAS E NOME DAS COLUNAS DA API LINXMOVIMENTO
for child in root.iter('*'):
    #print(child.tag, child.text)
    if child.tag == 'C':
        ncols_mov = len(child)
        print(len(child))
        print(ncols_mov)

In [ ]:
## QUEBRAR EM SUBLISTAS COM TAMANHO DAS COLUNAS N
for i in range(ncols_mov, len(lista_mov), ncols_mov):
    dados_mov.append(lista_mov[i: i + ncols_mov])

In [ ]:
dados_mov[0]

In [ ]:
df_mov = lista_mov[0:ncols_mov]

In [ ]:
## CRIAR DATAFRAME PARA ORGANIZAR E LIMPAR OS DADOS
dataframe = pd.DataFrame(dados_mov, columns=df_mov)

In [ ]:
## ELIMINA LINHAS CUJO STATUS DE PORTAL SEJA PORTAL ELIMINANDO ASSIM OS CABEÇALHOS
## QUE API ENVIA A CADA CONSULTA DE CNPJ
dataframe.drop(dataframe[dataframe.portal=='portal'].index, inplace=True)

In [ ]:
dataframe.head()

In [ ]:
## EXPORTAR PARA ARQUIVOCSV, CONFERENCIA E RETENÇÃO DOS DADOS
print(str(dt_10d)+'_'+str(ontem)+'_linxmovimento.csv')
dataframe.to_csv(str(dt_10d)+'_'+str(hoje)+'_linxmovimento.csv', index=False)

In [ ]:
## PARA TESTAR
## EXPORTAR PARA ARQUIVOCSV, CONFERENCIA E RETENÇÃO DOS DADOS
dataframe.to_csv('OutubroATE16_linxmovimento.csv', index=False)
## PARA TESTAR

##### FIM - CONSUMO DOS CUPONS DE VENDAS DAS LOJAS

##### INICIO - TRANSFORMAÇÃO E LIMPEZA DOS DADOS

In [ ]:
## LER DADOS DO ARQUIVO PARA TESTAR!!
#dataframe = pd.read_csv('2019-09-25_linxmovimento.csv', sep=',', low_memory=False, dtype=str, encoding='utf-8')

In [ ]:
##  CRIAR UM NOVO DATAFRAME DO DATAFRAME COM TODAS AS COLUNAS, SELECIONANDO APENAS AS COLUNAS QUE SERÃO UTILIZADAS
df = dataframe[['portal','cnpj_emp','transacao','documento','chave_nf','ecf','numero_serie_ecf','modelo_nf','data_documento','data_lancamento','codigo_cliente','serie','desc_cfop','id_cfop','cod_vendedor','quantidade','preco_custo','valor_liquido','desconto','valor_total','total_dinheiro','total_cheque','total_cartao','total_crediario','total_convenio','operacao','tipo_transacao','cod_produto','cod_barra','cancelado','excluido','soma_relatorio','identificador','preco_unitario','hora_lancamento','natureza_operacao','cod_sefaz_situacao','desc_sefaz_situacao','protocolo_aut_nfe','total_cheque_prazo','cod_natureza_operacao','preco_tabela_epoca','desconto_total_item']].reset_index()

In [ ]:
df.head()

In [ ]:
df.index

In [ ]:
## SETAR NOVO INDICE O DATAFRAME
#df.set_index('index', inplace=True)

In [ ]:
## DROPAR COLUNA DO DATAFRAME INDEX CRIADO, PELO REINDEX()
df.drop('index', inplace=True, axis=1)

In [ ]:
# REMOVER TRAÇOS NEGATIVOS EM VALOR_TOTAL PARA FAZER TODA DEVOLUÇÃO NEGATIVA EM VALOR_TOTAL
df.loc[0:, 'valor_total'] = df['valor_total'].str.replace('-', '')

In [ ]:
## LIMPAR CARACTERES DA COLUNA COD_NATUREZA_OPERACAO
df.loc[0:, 'cod_natureza_operacao'] = df['cod_natureza_operacao'].str.strip()

In [ ]:
## CARREGAR UMA TUPLA COM O INDEX ONDE NA COLUNA COD_NATUREZA_OPERACAO = 1.201 PARA ALTERAR PARA NEGATIVO O VALOR_TOTAL 
lista_devolucao = []
for row in df.itertuples():
    if row.cod_natureza_operacao == '1.201':
        lista_devolucao.append(row.Index)

In [ ]:
## TRANSFORMAR EM NEGATIVO VALOR_TOTAL E QUANTIDADE DA TUPLA LISTA_DEVOLUCAO
df.loc[lista_devolucao, 'valor_total'] = df['valor_total'].map(lambda x: "-{}".format(x))
df.loc[lista_devolucao, 'quantidade'] = df['quantidade'].map(lambda x: "-{}".format(x))

In [ ]:
## PREENCHER NAN
#df.fillna('NULL', inplace=True) #TOTAL, TUDO
values = {'chave_nf': ' ', 'numero_serie_ecf':' ', 'modelo_nf': '0','desc_cfop': ' ', 'id_cfop': '0', 'tipo_transacao': ' ', 'valor_total': '0', 'cod_barra': '0', 'cod_sefaz_situacao': '0', 'desc_sefaz_situacao': ' ', 'protocolo_aut_nfe': '0', 'desconto_total_item': '0'}
df = df.fillna(value=values)

In [ ]:
## RETIRANDO ACENTOS PARA TABELA ASCCI NAS COLUNAS
df.loc[0:, 'desc_cfop'] = df['desc_cfop'].apply(lambda x: normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))
df.loc[0:, 'natureza_operacao'] = df['natureza_operacao'].apply(lambda x: normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

In [ ]:
## REMOVER TRACINHOS
df.loc[0:, 'data_documento'] = df['data_documento'].str.replace('-', '')
df.loc[0:, 'data_lancamento'] = df['data_lancamento'].str.replace('-', '')

In [ ]:
## ACERTANDO FORMATO DE DATA
df.loc[0:, 'data_documento'] = df['data_documento'].map(lambda d: "{}".format(d[0:8]))
df.loc[0:, 'data_lancamento'] = df['data_lancamento'].map(lambda d: "{}".format(d[0:8]))

In [ ]:
## CRIAR LISTA DE COLUNAS COLUNAS QUE DEVEM SER DO TIPO INT
cols_int = ['portal','cnpj_emp','transacao','documento','ecf','modelo_nf','codigo_cliente','id_cfop','cod_vendedor','cod_produto','cod_barra','cod_sefaz_situacao','protocolo_aut_nfe']

In [ ]:
## CONVERTER PARA INT LISTA DE COLUNAS QUE DEVEM SER DO TIPO INT
df[cols_int] = df[cols_int].apply(lambda x: pd.to_numeric(x.astype('int64'), errors='coerce'))

In [ ]:
## PERCORRER COLUNA QUANTIDADE, PRIMEIRO FLOAT, DEPOIS ROUND() DEPOIS INT
df['quantidade'] = df['quantidade'].astype('float64', errors='ignore')
df['quantidade'] = df['quantidade'].round()
df['quantidade'] = df['quantidade'].astype('int64')

In [ ]:
## CRIAR LISTA DE COLUNAS COLUNAS QUE DEVEM SER DO TIPO FLOAT
cols_float = ['preco_custo','valor_liquido','desconto','valor_total','total_dinheiro','total_cheque','total_cartao','total_crediario','total_convenio','preco_unitario','total_cheque_prazo','preco_tabela_epoca','desconto_total_item']

In [ ]:
## VERIFICAR POR DADOS NAN NO DATAFRAME
df.isnull().sum() != 0

In [ ]:
## ENCONTRAR VALOR VAZIO EM TIPO_TRANSACAO OU ERROS
df.loc[(df['tipo_transacao']).isnull()]

In [ ]:
## CONVERTER PARA FLOAT COLUNAS DE PREÇOS
df[cols_float] = df[cols_float].apply(lambda x: pd.to_numeric(x.astype(str), errors='coerce'))

In [ ]:
df.head(1000)

In [ ]:
df.loc[607261:607263]

In [ ]:
df.index

In [ ]:
df_list = []

In [ ]:
df_list = df.values.tolist()

In [ ]:
print(len(df_list))

In [ ]:
#df_list[607262]

In [ ]:
#df_list[607262][28]

In [ ]:
#df_list[607262][28] = 7909300212256

In [ ]:
#df_list2 = df_list[600000:]

In [ ]:
#df_list2[7262]

##### FIM - TRANSFORMAÇÃO E LIMPEZA DOS DADOS

In [ ]:
print(dias_10)
print(ontem_10)

##### INICIO - DELETAR DA BASE 10 DIAS ANTERIORES - PARA INCLUIR ATUALIZAÇÕES

In [ ]:
## STRING DE CONEXÃO ORACLE
connect = cx_Oracle.connect('USER/PASS@SERVER/SID_DB')

In [ ]:
## DELETAR NA BASE DE DADOS OS 10 DIAS ANTERIORES
try:
    #statement = 'DELETE FROM DW_LINX WHERE DATA_LANCAMENTO BETWEEN '+str(dias_10)+' AND '+str(ontem_10)+''
    statement = 'DELETE FROM DW_LINX WHERE DATA_LANCAMENTO BETWEEN 20190926 AND 20191231'
    cursor = connect.cursor()
    cursor.execute(statement)
    connect.commit()
except cx_Oracle.DatabaseError as e:
    errorObj, = e.args
    print("ERRO!! ", errorObj.message)
    connect.rollback()
    cursor.close()
    connect.close()

cursor.close()
connect.close()
print('DELETADO 10 DIAS ANTERIORES A PARTIR DE ONTEM!!')

##### FIM - DELETAR DA BASE 10 DIAS ANTERIORES - PARA INCLUIR ATUALIZAÇÕES

In [ ]:
df_list[lista_devolucao[0]]

##### INICIO - INSERIR DADOS DE 10 DIAS CONSULTADOS NA API NA TABELA DW_LINX

In [ ]:
## STRING DE CONEXÃO ORACLE
connect = cx_Oracle.connect('USER/PASS@SERVER/SID_DB', encoding='utf-8', nencoding='utf-8')

In [ ]:
## INSERIR NA BASE DE DADOS - METODO MATRIZ
rows = df_list
cursor = connect.cursor()
cursor.bindarraysize = 43
cursor.setinputsizes(int,int,int,int,200,int,50,int,8,8,int,50,300,int,int,int,float,float,float,float,\
                     float,float,float,float,float,2,1,int,int,1,1,1,50,float,8,50,int,50,int,float,15,float,float)
statement = 'INSERT INTO DW_LINX (portal,cnpj_emp,transacao,documento,chave_nf,ecf,numero_serie_ecf,\
modelo_nf,data_documento,data_lancamento,codigo_cliente,serie,desc_cfop,id_cfop,cod_vendedor,quantidade,\
preco_custo,valor_liquido,desconto,valor_total,total_dinheiro,total_cheque,total_cartao,total_crediario,\
total_convenio,operacao,tipo_transacao,cod_produto,cod_barra,cancelado,excluido,soma_relatorio,identificador,\
preco_unitario,hora_lancamento,natureza_operacao,cod_sefaz_situaca,desc_sefaz_situacao,protocolo_aut_nfe,\
total_cheque_prazo,cod_natureza_operacao,preco_tabela_epoca,desconto_total_item) \
VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,\
:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42,:43)'
for i in range(0, len(rows), 10000):
    try:
        cursor.executemany(statement, rows[i:i+10000])
        connect.commit()
        print('Range :::', i ,'Até:::', i+9999 ,'::: INSERIDO/COMMIT COM SUCESSO :::')
    except cx_Oracle.DatabaseError as e:
        errorObj, = e.args
        print("NO RANGE ", i ,"A LINHA ", cursor.rowcount, "ESTÁ COM ERRO ", errorObj.message)
        connect.rollback()
        cursor.close()
        connect.close()
        break

cursor.close()
connect.close()
print('INSERÇÃO DOS DADOS OK!!')

##### FIM - INSERIR DADOS DE 10 DIAS CONSULTADOS NA API NA TABELA DW_LINX